In [1]:
import time
import pickle
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
from labels import label_datetimes
from scipy.interpolate import UnivariateSpline

In [7]:
def get_ts(date_str):
    ts = time.mktime(datetime.strptime(
        date_str, "%Y-%m-%d %H:%M:%S").timetuple())
    return int(ts)

In [8]:
def get_values_between_dates(matrix, sensor_idx, sensor_data, start_date, end_date):
    start_ts = get_ts(start_date)
    end_ts = get_ts(end_date)
    t = np.linspace(start_ts, end_ts, 100)
    result = {"t": t, "hps": []}
    for hp in range(10):
        df = sensor_data[matrix][sensor_idx][hp]
        df_interval = df[(df["Real time clock"] >= start_ts - 100)
                         & (df["Real time clock"] <= end_ts + 100)]

        rtc = df_interval["Real time clock"].values
        gas = df_interval["Resistance Gassensor"].values

        interpolation_func = UnivariateSpline(rtc, gas, s=0)

        y = interpolation_func(t)
        result["hps"].append(y)

    return result

In [10]:
DAY = 13
label = next((i for i in label_datetimes if i["day"] == DAY), None)

with open(f"data/day_{DAY}/raw.pkl", "rb") as f:
    raw_data = pickle.load(f)

interp_data = {"air": [], "sample": []}
for matrix in interp_data:
    for s_idx in range(8):
        s_data = get_values_between_dates(
            matrix,
            s_idx,
            raw_data,
            label["start"],
            label["end"])

        interp_data[matrix].append(s_data)

with open(f"data/day_{DAY}/interp.pkl", "wb") as f:
    pickle.dump(interp_data, f)